In [1]:
import pandas as pd

In [2]:
import json
with open('raw_data/waseem_data (1).json') as f:
  waseem = json.load(f)

In [3]:
df = pd.DataFrame.from_dict(waseem, orient="index")

In [4]:
df.head(2)

,raw_text,raw_label,label_map
572332655397629952,these girls are the equivalent of the irritati...,racism,1
572341498827522049,drasko they didn't cook half a bird you idiot ...,none,0


In [5]:
posts = df['raw_text'].to_list()
raw_labels = df['raw_label'].to_list()

In [6]:
set(raw_labels)

{'none', 'racism', 'sexism'}

In [7]:
labels = []
for i in raw_labels:
    if i in ['racism', 'sexism']:
        labels.append('Hate')
    else:
        labels.append('NonHate')

In [8]:
df = pd.DataFrame({'final_posts':posts, 
                              'final_labels':labels, 
                            })

In [10]:
import cleansetext
from cleansetext.pipeline import Pipeline
from cleansetext.steps import *
from nltk.tokenize import TweetTokenizer

tk = TweetTokenizer()

# Create a pipeline with a list of preprocessing steps
pipeline = Pipeline([
    RemoveEmojis(),
    RemoveAllPunctuations(),
    RemoveTokensWithOnlyPunctuations(),
    ReplaceURLsandHTMLTags(),
    ReplaceUsernames(),
    RemoveWhiteSpaceOrChunksOfWhiteSpace()
], track_diffs=False)

def apply_preprocessing(text):
    text = tk.tokenize(text)
    text = pipeline.process(text)
    return " ".join(text)

In [11]:
df['final_posts'] = df['final_posts'].apply(lambda x: apply_preprocessing(x))

In [12]:
from sklearn.model_selection import train_test_split

df_waseem_train, df_waseem_test = train_test_split(df, random_state=42, test_size=0.25)

In [13]:
SAVE_PATH = 'prepared_data/'

df_waseem_train.to_csv(SAVE_PATH + 'df_waseem_train.csv')
df_waseem_test.to_csv(SAVE_PATH + 'df_waseem_test.csv')